<a href="https://colab.research.google.com/github/santamav/Sistemes-Intelligents_Big-Data-Analytics/blob/master/7th_Deliverable_Vicent_Santamarta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DELIVERABLE: FINE-TUNING AND ADAPTATION OF LANGUAGE MODELS

In this exercise the aim is to perform transfer learning from English to Spanish in the domain of sentiment analysis in opinions about movies.
It is recommended to use a cross-lingual model in order to facilitate the learning transfer.

- `xlm-roberta-base`
- `xlm-roberta-large`

Choose between transformers or simpletransformers

In [ ]:
!pip install transformers[torch]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 29.6 MB/s eta 0:00:00
ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


Get the data of opinions about movies from IMDB dataset (parquet format)

In [ ]:
!wget -O imdb_train.parquet "https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/train/0000.parquet"
!wget -O imdb_test.parquet  "https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/test/0000.parquet"

--2023-11-15 21:56:36--  https://huggingface.co/datasets/imdb/resolve/refs%2Fconvert%2Fparquet/plain_text/train/0000.parquet
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.74, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/imdb/d01f1e2b447fc7c7bffc1b03d64b5394e9fc9b1aac78a6548553f7c74bf69738?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1700343759&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDM0Mzc1OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9pbWRiL2QwMWYxZTJiNDQ3ZmM3YzdiZmZjMWIwM2Q2NGI1Mzk0ZTlmYzliMWFhYzc4YTY1NDg1NTNmN2M3NGJmNjk3Mzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=d5aF78nLM3hj0dij9MLY-1pkioBntkqFbJvPsLBDpXhWrdwReyv-oO3MYSI0Yg6ygiyZwow%7E

In [ ]:
!ls

imdb_test.parquet  imdb_train.parquet  sample_data


# PART I: Training with the English dataset

## Loading, exploring and transforming data for tranformers

In [ ]:
import pandas as pd
# Load the training dataset
df_train = pd.read_parquet("imdb_train.parquet")
# Load the test set
test_df = pd.read_parquet("imdb_test.parquet")

In [ ]:
df_train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [ ]:
df_train["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(test_df)

## Prepare the Trainer from a cross-lingual pretrained model

### Get the tokenizer and the classification model (from_pretrained)

In [ ]:
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
# Model that we are going to fine tune
model_name = "xlm-roberta-base" # @param ["xlm-roberta-base", "xlm-roberta-large"]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name)


(…)lm-roberta-base/resolve/main/config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

(…)ase/resolve/main/sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

(…)roberta-base/resolve/main/tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

In [ ]:
ratings_encoded = train_dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
test_encoded = test_dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
ratings_encoded

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

## Split train-validation-test

In [ ]:
from sklearn.model_selection import train_test_split
# Percentage for the validation set
val_size = 0.2 # @param {type:"slider", min:0, max:1, step:0.1}
use_seed = True #@param {type: "boolean"}
# Separate the data in training set and the validation set
if(use_seed):
  seed = 200 #@param {type:"number"}
  train_data = ratings_encoded.train_test_split(test_size=val_size, seed=seed)
else:
  train_data = ratings_encoded.train_test_split(test_size=val_size)

# Split the datasets and shuffle the data in the test set
train_ds = train_data['train']
val_ds = train_data['test']
test_ds = test_encoded.shuffle(seed=42)

## Fine-tune the model

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
batch_size = 12

training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=1, #10
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="steps", #"epochs"
                                  #logging_steps = len(emotions["train"]) // batch_size
                                  fp16 = True, #speed optimization
                                  #eval_steps = 50,
                                  #save_strategy="no",
                                  disable_tqdm=False)

In [ ]:
# Prepare the trainer
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=train_ds,
                  eval_dataset=val_ds)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
500,0.402600,0.259021,0.919400,0.919400
1000,0.270900,0.251948,0.927000,0.927000
1500,0.235300,0.228739,0.931400,0.931398


TrainOutput(global_step=1667, training_loss=0.29311529433005, metrics={'train_runtime': 1147.8544, 'train_samples_per_second': 17.424, 'train_steps_per_second': 1.452, 'total_flos': 5262221107200000.0, 'train_loss': 0.29311529433005, 'epoch': 1.0})

## Evaluate the model with the test dataset (English)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

def evaluate_model(trainer, test_set):
  prediction = trainer.predict(test_ds)
  true_labels = test_ds['label']
  predicted_labels = prediction.predictions.argmax(axis=1)
  labels = ['positive', 'negative']

  accuracy = accuracy_score(true_labels, predicted_labels)

  cm = confusion_matrix(true_labels, predicted_labels)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot()
  disp.ax_.set_title(f"Test Accuracy: {accuracy}")

In [ ]:
evaluate_model(trainer, test_ds)

KeyboardInterrupt: ignored

### Save the model

In [ ]:
import os
if not os.path.isdir("model"):
   os.mkdir("model")

model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/tokenizer.json')

In [ ]:
!tar -czf model_emotions.tgz model/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp model_emotions.tgz "/content/drive/MyDrive/Colab Notebooks/Move_ratings_sentiment_analysis"

# Part II: Adaptation to Spanish dataset (criticas_pelis)

Using the same model we have evaluated in the previous part, we are going to re-train it with the Spanish examples.

Take care of using the same labels in the Spanish dataset (0=negative, 1=positive). Neutral opinions (rate=3) will be discarded from the training dataset, but we can use them afterwards to predict their polarity with the resulting model.

## Load, explore and transform the Spanish dataset of Critics

In [ ]:
# Load the spanish data set
!wget https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
!gunzip criticas_pelis.csv.gz

In [ ]:
df_es = pd.read_csv("criticas_pelis.csv", names=["Row", "File", "Text", "Rating"])
df_es.head()

In [ ]:
# Transform the labels to 1 > 3; 0 < 3; discard = 3
def rating2label(x):
  if x < 3: return 0
  if x > 3: return 1
  return x

df_es['labels'] = df_es['Rating'].apply(rating2label)

df_es_3stars = df_es[df_es['Rating'] == 3]
df_es = df_es [df_es.Rating != 3]

df_es.drop(columns=['Rating'], inplace=True)

df_es.rename(columns={"Text": "text"})

df_es.drop(columns=['Row','File'], inplace=True)

In [ ]:
df_es['labels'].value_counts()/df_es.shape[0]

## Prepare the Trainer from the previous trained model

In [ ]:
# Prepare the model from the previous trained model
tokenizer = AutoTokenizer.from_pretrained("./model")
model = XLMRobertaForSequenceClassification.from_pretrained("./model")


In [ ]:
# Transform to dataset and tokenize
es_dataset = Dataset.from_pandas(df_train)
es_encoded = es_dataset.map(tokenize, batched=True, batch_size=200)

In [ ]:
# Separate in train-val-test
#@markdown How much of the set goes to training and how much to test:
train_size = 0.65 # @param {type:"slider", min:0, max:1, step:0.1}
# @markdown The val size is a percentage of the training set:
val_size = 0.2 # @param {type:"slider", min:0, max:1, step:0.1}

if(use_seed):
  es_train_test_data = es_encoded.train_test_split(test_size=(1-train_size), seed=seed)
  es_train_val_data = es_train_test_data['train'].train_test_split(test_size=val_size, seed=seed)
else:
  es_train_test_data = es_encoded.train_test_split(test_size=(1-train_size))
  es_train_val_data = es_train_test_data['train'].train_test_split(test_size=val_size)

# Split the datasets and shuffle the data in the test set
es_train_ds = es_train_val_data['train']
es_val_ds = es_train_val_data['test']
es_test_ds = es_train_test_data['test'].shuffle(seed=42)

In [ ]:
es_train_ds

In [ ]:
es_val_ds

In [ ]:
es_test_ds

## Fine-tune the model

In [ ]:
batch_size = 12

training_args_es = TrainingArguments(output_dir="results",
                                  num_train_epochs=1, #10
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="steps", #"epochs"
                                  #logging_steps = len(emotions["train"]) // batch_size
                                  fp16 = True, #speed optimization
                                  #eval_steps = 50,
                                  #save_strategy="no",
                                  disable_tqdm=False)

In [ ]:
# Prepare the trainer
trainer_es = Trainer(model=model,
                  args=training_args_es,
                  compute_metrics=compute_metrics,
                  train_dataset=es_train_ds,
                  eval_dataset=es_val_ds)

In [ ]:
trainer_es.train()

## Evaluate the model with the test dataset (Spanish)

In [ ]:
evaluate_model(trainer_es, es_test_ds)

## Evaluate again the model on the English test dataset

In [ ]:
evaluate_model(trainer_es, test_ds)

### Evaluate with the 3 stars ratings

In [ ]:
# Transform to dataset and tokenize
es_3stars_dataset = Dataset.from_pandas(df_es_3stars)
es_3stars_encoded = es_3stars_dataset.map(tokenize, batched=True, batch_size=200)

In [ ]:
evaluate_model(trainer_es, es_3stars_encoded)